In [1]:
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver # 브라우저를 조작하는 모듈
from selenium.webdriver.chrome.service import Service # 크롬 드라이버의 시작과 중지를 담당하는 클래스
from selenium.webdriver.common.by import By # 엘리먼트를 어떤 방식으로 선택할 지에 대한 상수
from webdriver_manager.chrome import ChromeDriverManager # 크롬 드라이버를 관리
import pandas as pd
from tqdm.auto import tqdm
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
DATA_PATH = "C:/Users/csjin/Desktop/crawling/data/"

In [3]:
def all_review(soup):
    review = soup.find('a', attrs={'data-nclick':'N=a:tab*s.srev'}).text
    review = int(review.split('쇼핑몰리뷰')[1].replace(',', ''))
    return review

In [4]:
def review_per_page(soup):
    _all = soup.find_all('span', attrs={'class':'reviewItems_average__0kLWX'})
    return len(_all)

In [5]:
def next_click(driver, i):
    selector = f"a[href='#'][role='button'][data-nclick='N=a:rev.page,r:{i}']"
    button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
    button.click()

In [6]:
def data_collect(soup, i):
    # 댓글 본문 수집
    content = soup.find_all('p', attrs = {'class' : 'reviewItems_text__XrSSf'})[i].text

    # 전체 수집
    text = soup.find_all('div', attrs={'class':'reviewItems_etc_area__3VUjt'})[i].text
    
    # 평점 수집
    score = int(text[2])
    
    # 날짜 수집
    p = re.compile('[0-9]+.[0-9]+.[0-9]+.')
    date = p.findall(text)[0]

    # 옵션 수집
    option = text.split(date)[1]
    
    data = [content, score, date, option]
    return data
    

In [7]:
def rating_click(driver, i):
    driver.find_element(By.CSS_SELECTOR, f'a[href="#"][data-nclick="N=a:rev.grd{i}"]').click()

In [8]:
def photo_click(driver):
    driver.find_element(By.CSS_SELECTOR, 'a[href="#"][data-nclick="N=a:rev.photoon"]').click()

In [9]:
def recent_click(driver):
    driver.find_element(By.CSS_SELECTOR, 'a[href="#"][data-nclick="N=a:rev.rec"]').click()

In [14]:
url_lst = ['https://search.shopping.naver.com/catalog/17448001091?query=%EB%8B%A5%ED%84%B0%EA%B7%B8%EB%A3%A8%ED%8A%B8&NaPm=ct%3Dlma7w5t4%7Cci%3D58b8717ff1fc41c5b92b16823e1d90bf64fe5c73%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dbd90b8aa2dba51a272492531b9ca15cfc8ac003f']

In [19]:
# 최신순으로 5점만
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # 전체 화면
options.add_argument("--incognito") # 시크릿 모드
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

for url in tqdm(url_lst, desc="outer", position=0):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "p.reviewItems_text__XrSSf")))
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    review = all_review(soup)
    
    page_num = (review // 20) + 1
    result = []
    
    recent_click(driver)
    
    time.sleep(0.2)
    
    rating_click(driver, 5)
    
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 1페이지 수집
    for i in range(review_per_page(soup)):
        data = data_collect(soup, i)
        result.append(data)
    
    # 2페이지 부터 수집
    for i in range(2, 101):
        try:
            next_click(driver, i)
        except:
            break
        time.sleep(0.2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        for j in range(review_per_page(soup)):
            data = data_collect(soup, j)
            result.append(data)

        time.sleep(0.5)

    results_df = pd.DataFrame(result)
    results_df.columns = ['content', 'score', 'date', 'option']

outer:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
# 포토&동영상 리뷰로 5점만
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # 전체 화면
options.add_argument("--incognito") # 시크릿 모드
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

for url in tqdm(url_lst, desc="outer", position=0):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "p.reviewItems_text__XrSSf")))
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    review = all_review(soup)
    
    page_num = (review // 20) + 1
    result = []
    
    photo_click(driver)
    
    time.sleep(0.2)
    
    rating_click(driver, 5)
    
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 1페이지 수집
    for i in range(review_per_page(soup)):
        data = data_collect(soup, i)
        result.append(data)
    
    # 2페이지 부터 수집
    for i in range(2, 101):
        try:
            next_click(driver, i)
        except:
            break
        time.sleep(0.2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        for j in range(review_per_page(soup)):
            data = data_collect(soup, j)
            result.append(data)

        time.sleep(0.5)

    results_df = pd.DataFrame(result)
    results_df.columns = ['content', 'score', 'date', 'option']
        

outer:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # 전체 화면
options.add_argument("--incognito") # 시크릿 모드
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

for url in tqdm(url_lst, desc="outer", position=0):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "p.reviewItems_text__XrSSf")))
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    review = all_review(soup)
    
    page_num = (review // 20) + 1
    result = []
    for i in range(1,6):
        rating_click(driver, i)
        
        time.sleep(0.2)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        # 1페이지 수집
        for i in range(review_per_page(soup)):
            data = data_collect(soup, i)
            result.append(data)
        
        # 2페이지 부터 수집
        for i in range(2, 101):
            try:
                next_click(driver, i)
            except:
                break
            time.sleep(0.2)
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            for j in range(review_per_page(soup)):
                data = data_collect(soup, j)
                result.append(data)
    
            time.sleep(0.5)

    results_df = pd.DataFrame(result)
    results_df.columns = ['content', 'score', 'date', 'option']
        
    

outer:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
results_df

,content,score,date,option
0,제품은 좋은데 배송이 만족스럽지 못했습니다,5,23.09.08.,닥터그루트 지성 두피용 샴푸 400ml: -포켓몬 디자인 상품 출고에 동의합니다.
1,제품은 항상 쓰던거 배송좋아요,5,23.09.07.,닥터그루트 지성 두피용 샴푸 400ml: -포켓몬 디자인 상품 출고에 동의합니다.
2,재구매요 쿨링감 있고요 파손 안되서와서 다행히에요,5,23.09.06.,
3,배송도 빠르고 저렴하게 사게되어서 좋네요,5,23.09.06.,닥터그루트 지성 두피용 샴푸 400ml: -포켓몬 디자인 상품 출고에 동의합니다.
4,탈모완화효과를 체감할 수 있는 샴푸!,5,23.09.06.,
...,...,...,...,...
1995,맛있고 배송도 빠르고 넘 넘 좋아요~,5,21.09.17.,
1996,역시 닥터그루트닙니당,5,21.09.17.,
1997,떡 잘지는 머리에 좋아요,5,21.09.16.,
1998,빠른배송 감사합니다 ~.^^,5,21.09.15.,


In [22]:
results_df.to_csv('C:/Users/csjin/Desktop/crawling/data/닥터그루트_2_recent.csv', index=False, encoding='utf-8-sig')